##  * * * * * * * * *  - : `Prefect Workflow` : -  * * * * * * * * *

### * `Task` :-  Build a Prefect Workflow and Auto Schedule it. Show the Prefect Dashboard with relevant outputs.

In [1]:
### Load the required libraries
%time

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

import re
from tqdm import tqdm, tqdm_notebook

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer

import nltk
nltk.download('omw-1.4')
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

Wall time: 0 ns


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\91994\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
def load_data(file_path):
    """
    Load data from a CSV file.
    """
    return pd.read_csv(file_path)

In [3]:
def split_inputs_output(data, inputs, output):
    """
    Split features and target variables.
    """
    X = data[inputs]
    y = data[output]
    return X, y

In [4]:
def split_train_test(X, y, test_size=0.2, random_state=42):
    """
    Split data into train and test sets.
    """
    return train_test_split(X, y, test_size=test_size, random_state=random_state)

In [5]:
def preprocess_data(X_train, X_test, y_train, y_test):
    """
    Pre-processing the data
    """
    def clean_text(data):
        """
        cleaning the data.
        """
        tqdm.pandas()
        lemmatizer = WordNetLemmatizer()
    
        # Removing special characters and digits
        sentence = re.sub(r"[^a-zA-Z0-9\s]", " ", data)
    
        # change sentence to lower case
        sentence = sentence.lower()

        # tokenize into words
        tokens = sentence.split()
    
        # remove stop words                
        clean_tokens = [t for t in tokens if not t in stopwords.words("english")]
    
        clean_tokens = [lemmatizer.lemmatize(word) for word in clean_tokens]
    
        sentence =  " ".join(clean_tokens)
    
        return sentence
    
    for col in X_train.columns:
        X_train[col] = X_train[col].apply(lambda doc: clean_text(doc))
    for col in X_test.columns:
        X_test[col] = X_test[col].apply(lambda doc: clean_text(doc))
    
    return X_train, X_test, y_train, y_test

In [6]:
def vectorizing_data(X_train, X_test, y_train, y_test):
    """
    vectorizing the data.
    """
    vector = CountVectorizer()
    X_train_vec = vector.fit_transform(X_train)
    X_test_vec = vector.transform(X_test)
    return X_train_vec, X_test_vec, y_train, y_test

In [7]:
def train_model(X_train_vec, y_train, hyperparameters):
    """
    Training the machine learning model.
    """
    clf = DecisionTreeClassifier(**hyperparameters)
    clf.fit(X_train_vec, y_train)
    return clf

In [8]:
def evaluate_model(model, X_train_vec, y_train, X_test_vec, y_test):
    """
    Evaluating the model.
    """
    y_train_pred = model.predict(X_train_vec)
    y_test_pred = model.predict(X_test_vec)

    train_score = metrics.accuracy_score(y_train, y_train_pred)
    test_score = metrics.accuracy_score(y_test, y_test_pred)
    
    return train_score, test_score

In [9]:
def workflow(data_path):
    DATA_PATH = data_path
    INPUTS = ['Review text']
    OUTPUT = 'Review Sentiment'
    HYPERPARAMETERS = {'max_features' : 1000, 'max_depth': 10}
    
    # Load data
    reviews = load_data(DATA_PATH)

    # Identify Inputs and Output
    X, y = split_inputs_output(reviews, INPUTS, OUTPUT)

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = split_train_test(X, y)
    
    #preprocessing the data
    X_train, X_test, y_train, y_test = preprocess_data(X_train, X_test, y_train, y_test)
    
    # vectorizing to data
    X_train_vec, X_test_vec, y_train, y_test = vectorizing_data(X_train['Review text'], X_test['Review text'], y_train, y_test)
    
    # Build a model
    model = train_model(X_train_vec, y_train, HYPERPARAMETERS)
    
    # Evaluation
    train_score, test_score = evaluate_model(model, X_train_vec, y_train, X_test_vec, y_test)
    
    print("Train Score:", train_score)
    print("Test Score:", test_score)

In [10]:
if __name__ == "__main__":
    workflow(data_path="data/data.csv")

Train Score: 0.9263281479307308
Test Score: 0.8996478873239436


### * * * * `Building a Prefect Workflow` * * * *

##### Step 1 - Import Prefect modules

##### Step 2 - Define Prefect Tasks

##### Step 3 - Define Prefect Flow

##### Step 5 - Run Prefect Flow

In [11]:
from prefect import task, flow

In [12]:
@task
def load_data(file_path):
    """
    Load data from a CSV file.
    """
    return pd.read_csv(file_path)


@task
def split_inputs_output(data, inputs, output):
    """
    Split features and target variables.
    """
    X = data[inputs]
    y = data[output]
    return X, y


@task
def split_train_test(X, y, test_size=0.2, random_state=42):
    """
    Split data into train and test sets.
    """
    return train_test_split(X, y, test_size=test_size, random_state=random_state)


@task
def preprocess_data(X_train, X_test, y_train, y_test):
    """
    Pre-processing the data
    """
    def clean_text(data):
        """
        cleaning the data.
        """
        tqdm.pandas()
        lemmatizer = WordNetLemmatizer()
    
        # Removing special characters and digits
        sentence = re.sub(r"[^a-zA-Z0-9\s]", " ", data)
    
        # change sentence to lower case
        sentence = sentence.lower()

        # tokenize into words
        tokens = sentence.split()
    
        # remove stop words                
        clean_tokens = [t for t in tokens if not t in stopwords.words("english")]
    
        clean_tokens = [lemmatizer.lemmatize(word) for word in clean_tokens]
    
        sentence =  " ".join(clean_tokens)
    
        return sentence
    
    for col in X_train.columns:
        X_train[col] = X_train[col].apply(lambda doc: clean_text(doc))
    for col in X_test.columns:
        X_test[col] = X_test[col].apply(lambda doc: clean_text(doc))
    
    return X_train, X_test, y_train, y_test


@task
def vectorizing_data(X_train, X_test, y_train, y_test):
    """
    vectorizing the data.
    """
    vector = CountVectorizer()
    X_train_vec = vector.fit_transform(X_train)
    X_test_vec = vector.transform(X_test)
    return X_train_vec, X_test_vec, y_train, y_test


@task
def train_model(X_train_vec, y_train, hyperparameters):
    """
    Training the machine learning model.
    """
    clf = DecisionTreeClassifier(**hyperparameters)
    clf.fit(X_train_vec, y_train)
    return clf


@task
def evaluate_model(model, X_train_vec, y_train, X_test_vec, y_test):
    """
    Evaluating the model.
    """
    y_train_pred = model.predict(X_train_vec)
    y_test_pred = model.predict(X_test_vec)

    train_score = metrics.accuracy_score(y_train, y_train_pred)
    test_score = metrics.accuracy_score(y_test, y_test_pred)
    
    return train_score, test_score

In [13]:
# Workflow

@flow(name="Decision Tree Training Flow")
def workflow():
    DATA_PATH = "data/data.csv"
    INPUTS = ['Review text']
    OUTPUT = 'Review Sentiment'
    HYPERPARAMETERS = {'max_features' : 1000, 'max_depth': 10}
    
    # Load data
    reviews = load_data(DATA_PATH)

    # Identify Inputs and Output
    X, y = split_inputs_output(reviews, INPUTS, OUTPUT)

    # Split data into train and test sets
    X_train, X_test, y_train, y_test = split_train_test(X, y)
    
    #preprocessing the data
    X_train, X_test, y_train, y_test = preprocess_data(X_train, X_test, y_train, y_test)
    
    # vectorizing to data
    X_train_vec, X_test_vec, y_train, y_test = vectorizing_data(X_train['Review text'], X_test['Review text'], y_train, y_test)
    
    # Build a model
    model = train_model(X_train_vec, y_train, HYPERPARAMETERS)
    
    # Evaluation
    train_score, test_score = evaluate_model(model, X_train_vec, y_train, X_test_vec, y_test)
    
    print("Train Score:", train_score)
    print("Test Score:", test_score)

In [14]:
if __name__ == "__main__":
    workflow()

16:06:55.348 | INFO    | prefect.engine - Created flow run 'heavy-yak' for flow 'Decision Tree Training Flow'

16:06:55.680 | INFO    | Flow run 'heavy-yak' - Created task run 'load_data-0' for task 'load_data'

16:06:55.696 | INFO    | Flow run 'heavy-yak' - Executing 'load_data-0' immediately...

16:06:56.071 | INFO    | Task run 'load_data-0' - Finished in state Completed()

16:06:56.167 | INFO    | Flow run 'heavy-yak' - Created task run 'split_inputs_output-0' for task 'split_inputs_output'

...

16:07:43.350 | INFO    | Flow run 'heavy-yak' - Executing 'evaluate_model-0' immediately...

16:07:43.626 | INFO    | Task run 'evaluate_model-0' - Finished in state Completed()

Train Score: 0.9241267977692985
Test Score: 0.9008215962441315


16:07:43.734 | INFO    | Flow run 'heavy-yak' - Finished in state Completed('All states completed.')

[Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `DataFrame`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `tuple`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `list`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `tuple`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersisted result of type `tuple`')),
 Completed(message=None, type=COMPLETED, result=UnpersistedResult(type='unpersisted', artifact_type='result', artifact_description='Unpersiste